## Paquetes Basicos

In [1]:
import json
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

## Separación de imágenes entre entrenamiento y prueba

In [2]:
# Transforma de formato Coco a formato Yolo
def coco_to_yolo(x, y, w, h, width, height):
    return [((2*x + w)/(2*width)), ((2*y + h)/(2*height)), w/width, h/height]

In [5]:
def split_data(data, test_size=0.2, random_state=42):
    images = data['images']
    ids = [img['id'] for img in images]
    
    train_ids, rest_ids = train_test_split(ids, test_size=test_size, random_state=random_state)

    test_ids, val_ids = train_test_split(rest_ids, test_size=0.5, random_state=random_state)
    
    train_data = {'images': [img for img in images if img['id'] in train_ids],
                  'annotations': [ann for ann in data['annotations'] if ann['image_id'] in train_ids]}
    
    test_data = {'images': [img for img in images if img['id'] in test_ids],
                'annotations': [ann for ann in data['annotations'] if ann['image_id'] in test_ids]}

    val_data = {'images': [img for img in images if img['id'] in val_ids],
                'annotations': [ann for ann in data['annotations'] if ann['image_id'] in val_ids]}
    
    return train_data, test_data, val_data

def preprocessing(partition: str, input_path: str, output_path: str, data: object):
    os.makedirs(f"{output_path}/images/{partition}", exist_ok=True)
    os.makedirs(f"{output_path}/labels/{partition}", exist_ok=True)

    count = 1
    for fileinfo in data['images']:
        origin = f"{input_path}/plaga_processed/{fileinfo['file_name']}"
        img_id = fileinfo['id']
        width = fileinfo['width']
        height = fileinfo['height']

        img_ann = get_img_ann(img_id)

        if os.path.exists(origin):
            items = []
            for ann in img_ann:
                label = ann['category_id'] - 1
                box = ann['bbox']
                xc, yc, w, h = coco_to_yolo(box[0], box[1], box[2], box[3], width, height)
                items.append(f"{label} {xc} {yc} {w} {h}")

            with open(f"{output_path}/labels/{partition}/img{count}.txt", "w") as f:
                for item in items:
                    f.write(f"{item}\n")
        else:
            origin = f"{input_path}/sin_plaga_processed/{fileinfo['file_name']}"

        destination = f"{output_path}/images/{partition}/img{count}.png"
        try:
            shutil.copy(origin, destination)
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        count += 1

def get_img_ann(image_id):
    img_ann = []
    isFound = False
    for ann in data['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None

input_path = r"../data/raw"
output_path = r"../data/raw/dataYolo" 

# Cargar datos desde el archivo JSON
f = open(f"{input_path}/aguacate_data.json")
data = json.load(f)
f.close()

# Dividir datos en conjuntos de entrenamiento, prueba y validación
train_data, test_data, val_data = split_data(data, test_size=0.2, random_state=42)

# Procesar conjuntos de entrenamiento, prueba y validación
preprocessing("train", input_path, output_path, train_data)
preprocessing("test", input_path, output_path, test_data)
preprocessing("validation", input_path, output_path, val_data)

../data/raw/plaga_processed/Heilipus (1).png
../data/raw/plaga_processed/Heilipus (10).png
../data/raw/plaga_processed/Heilipus (101).png
../data/raw/plaga_processed/Heilipus (102).png
../data/raw/plaga_processed/Heilipus (103).png
../data/raw/plaga_processed/Heilipus (104).png
../data/raw/plaga_processed/Heilipus (105).png
../data/raw/plaga_processed/Heilipus (106).png
../data/raw/plaga_processed/Heilipus (107).png
../data/raw/plaga_processed/Heilipus (109).png
../data/raw/plaga_processed/Heilipus (11).png
../data/raw/plaga_processed/Heilipus (110).png
../data/raw/plaga_processed/Heilipus (111).png
../data/raw/plaga_processed/Heilipus (112).png
../data/raw/plaga_processed/Heilipus (113).png
../data/raw/plaga_processed/Heilipus (114).png
../data/raw/plaga_processed/Heilipus (115).png
../data/raw/plaga_processed/Heilipus (116).png
../data/raw/plaga_processed/Heilipus (117).png
../data/raw/plaga_processed/Heilipus (118).png
../data/raw/plaga_processed/Heilipus (119).png
../data/raw/plaga

## Entrenamiento y Evaluación del Modelo YOLO

In [9]:
!pip install ultralytics
from ultralytics import YOLO

In [12]:
model = YOLO("yolov8s.pt")
model.train(data="train.yaml", epochs=10)

New https://pypi.org/project/ultralytics/8.2.67 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.227 🚀 Python-3.11.5 torch-2.1.2+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=train.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_

train: Scanning D:\proyectoGrado\mlops_project\data\raw\dataYolo\labels\train... 555 images, 40 backgrounds, 0 corrupt: 100%|██████████| 595/595 [00:02<00:00, 262.89it/s]

train: New cache created: D:\proyectoGrado\mlops_project\data\raw\dataYolo\labels\train.cache



val: Scanning D:\proyectoGrado\mlops_project\data\raw\dataYolo\labels\validation... 68 images, 7 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<00:00, 227.17it/s]

val: New cache created: D:\proyectoGrado\mlops_project\data\raw\dataYolo\labels\validation.cache


...

                   all         75         68      0.981      0.985      0.989      0.729



10 epochs completed in 2.339 hours.
Optimizer stripped from runs\detect\train5\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train5\weights\best.pt, 22.5MB

Validating runs\detect\train5\weights\best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.11.5 torch-2.1.2+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:34<00:00, 11.66s/it]


                   all         75         68      0.981      0.985      0.989      0.729
Speed: 3.3ms preprocess, 300.9ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train5
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=False'


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002878011EA90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

## Evaluación de desempeño (conjunto de validación)

In [33]:
metrics = model.val(data="train.yaml", split="val")
matriz_confusion = metrics.confusion_matrix.matrix

# Extraer TP, FP, FN, TN
TP = matriz_confusion[0, 0]
FP = matriz_confusion[0, 1]
FN = matriz_confusion[1, 0]
TN = matriz_confusion[1, 1]

# Calcular el Accuracy, Precision y Recall
accuracy = (TP + TN) / (TP + FP + FN + TN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print(f'Accuracy al final de la ultima época: {accuracy*100}')
print(f'Precision al final de la ultima época: {precision*100}')
print(f'Recall al final de la ultima época: {recall*100}')

Ultralytics YOLOv8.0.227 🚀 Python-3.11.5 torch-2.1.2+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)


val: Scanning D:\proyectoGrado\mlops_project\data\raw\dataYolo\labels\validation.cache... 68 images, 7 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:38<00:00,  7.69s/it]


                   all         75         68      0.981      0.985      0.991       0.73
Speed: 2.6ms preprocess, 329.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train54
Accuracy al final de la ultima época: 95.71428571428572
Precision al final de la ultima época: 97.10144927536231
Recall al final de la ultima época: 98.52941176470588


## Evaluación de desempeño (conjunto de prueba)

In [34]:
metrics = model.val(data="train.yaml", split="test")
matriz_confusion = metrics.confusion_matrix.matrix

# Extraer TP, FP, FN, TN
TP = matriz_confusion[0, 0]
FP = matriz_confusion[0, 1]
FN = matriz_confusion[1, 0]
TN = matriz_confusion[1, 1]

# Calcular el Accuracy, Precision y Recall
accuracy = (TP + TN) / (TP + FP + FN + TN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print(f'Accuracy al final de la ultima época: {accuracy*100}')
print(f'Precision al final de la ultima época: {precision*100}')
print(f'Recall al final de la ultima época: {recall*100}')

Ultralytics YOLOv8.0.227 🚀 Python-3.11.5 torch-2.1.2+cpu CPU (Intel Core(TM) i5-9300H 2.40GHz)


val: Scanning D:\proyectoGrado\mlops_project\data\raw\dataYolo\labels\test.cache... 71 images, 3 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:35<00:00,  7.19s/it]


                   all         74         71      0.933      0.976      0.985      0.661
Speed: 2.5ms preprocess, 306.5ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train55
Accuracy al final de la ultima época: 93.24324324324324
Precision al final de la ultima época: 95.83333333333334
Recall al final de la ultima época: 97.1830985915493
